In [1]:
import os
os.environ['PYTHONPATH'] += '/root/PONI'

In [2]:
import sys
sys.path.append('/root/PONI')

In [3]:
import habitat
import numpy as np
from habitat_map.env_orb import Env
from habitat.sims.habitat_simulator.actions import HabitatSimActions
from greedy_path_follower_agent_policy_frontier import GreedyPathFollowerAgent
from poni_agent import PoniAgent
from habitat.config.default import get_config, patch_config
from omegaconf import OmegaConf
from habitat_map.utils import draw_top_down_map
from skimage.io import imsave
from PIL import Image
from time import time
import cv2
import os
import yaml

Failed to import TensorRT, no inference acceleration
Failed to import TensorRT, no inference acceleration


In [4]:
import matplotlib.pyplot as plt
from io import BytesIO
from tqdm import tqdm_notebook
%matplotlib inline

In [5]:
class HabitatRunner():
    def __init__(self):
        #rospy.init_node('habitat_runner')
        fin = open('config_poni.yaml', 'r')
        config = yaml.safe_load(fin)
        fin.close()

        # Now define the config for the sensor
        habitat_path = '/home/kirill/habitat-lab/data'
        task_config = get_config(config['task']['config'])
        OmegaConf.set_readonly(task_config, False)
        #task_config.defrost()
        #task_config.DATASET.DATA_PATH = os.path.join(habitat_path, 'datasets/objectnav_hm3d_v1/val/val.json.gz')
        #task_config.habitat.environment.iterator_options.shuffle = False
        #task_config.TASK.AGENT_POSITION_SENSOR = habitat.Config()
        #task_config.TASK.AGENT_POSITION_SENSOR.TYPE = "position_sensor"
        #task_config.TASK.AGENT_POSITION_SENSOR.ANSWER_TO_LIFE = 42
        #task_config.TASK.SENSORS.append("AGENT_POSITION_SENSOR")
        #task_config.habitat.simulator.AGENT_0.SENSORS.append("SEMANTIC_SENSOR")
        #task_config.SIMULATOR.SCENE_DATASET = os.path.join(habitat_path, "scene_datasets/hm3d/hm3d_annotated_basis.scene_dataset_config.json")
        #task_config.TASK.MEASUREMENTS.append("TOP_DOWN_MAP")
        #task_config.TASK.SENSORS.append("HEADING_SENSOR")
        #task_config.freeze()

        # Initialize environment
        self.env = Env(config=task_config)
        #env = env_orb.env(config=config)
        print('Environment created')

        # Initialize metric counters
        self.eval_episodes = config['task']['eval_episodes']
        #self.eval_episodes = self.eval_episodes[:1]
        self.successes = []
        self.spls = []
        self.softspls = []
        self.fake_finishes = 0

        # initialize agent
        #path_follower_config = config['path_follower']
        #agent_type = path_follower_config['type']
        agent_type = 'greedy'
        if agent_type == 'greedy':
            self.agent = GreedyPathFollowerAgent(task_config)
        elif agent_type == 'poni':
            self.agent = PoniAgent(task_config)
        else:
            print('AGENT TYPE {} IS NOT DEFINED!!!'.format(agent_type))
            self.agent = None

        self.top_down_map_save_path = config['task']['top_down_map_save_path']
        self.last_pic_save_path = config['task']['last_pic_save_path']


    def run_episode(self, ii):
        print('Run episode')
        observations = self.env.reset(i=ii)
        print('Episode number', ii)
        objectgoal = observations['objectgoal'][0]
        self.objectgoal = objectgoal
        objectgoal_name = {v: k for k, v in self.env.task._dataset.category_to_task_category_id.items()}[objectgoal]
        print('Objectgoal:', objectgoal_name)

        self.agent.reset()

        # Run the simulator with agent
        #step_start_time = rospy.Time.now()
        observations = self.env.step(HabitatSimActions.move_forward)

        finished = False
        cost_values = []
        agent_trajectory = []
        #while not rospy.is_shutdown() and not self.env.episode_over:
        while not self.env.episode_over:
            #cost_values.append(self.agent.exploration.goal_cost)
            robot_x, robot_y = observations['gps']
            robot_y = -robot_y
            agent_trajectory.append([robot_x, robot_y])
            action = self.agent.act(observations)
            observations = self.env.step(action)
            if action == HabitatSimActions.stop:
                finished = True
                break
        #critic_values = self.agent.critic_values

        # Calculate and show metrics
        metrics = self.env.task.measurements.get_metrics()
        print('METRICS:', metrics)

        if finished and metrics['success'] == 0:
            self.fake_finishes += 1
        print('Success:', metrics['success'])
        print('SPL:', metrics['spl'])
        print('SoftSPL:', metrics['soft_spl'])
        self.successes.append(metrics['success'])
        self.spls.append(metrics['spl'])
        self.softspls.append(metrics['soft_spl'])
        print('Average success:', np.mean(self.successes))
        print('Average SPL:', np.mean(self.spls))
        print('Average softSPL:', np.mean(self.softspls))
        print('Number of false goal detections:', self.fake_finishes)

In [6]:
gif_save_dir = 'fbe_results/attempt1'

In [7]:
%matplotlib agg
import gc
from moviepy.editor import ImageSequenceClip

def get_img_from_fig(fig, dpi=180):
    buf = BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def make_gif(maps, rgbs, depths, robot_poses, goal_coords, action_track, episode_number, objectgoal):
    frames = []
    for i in tqdm_notebook(range(len(maps))):
        fig = plt.figure(figsize=(10, 3))
        plt.subplot(1, 3, 1)
        plt.imshow(rgbs[i])
        plt.subplot(1, 3, 2)
        plt.imshow((depths[i] - 0.5) / 4.5, cmap='gray')
        plt.subplot(1, 3, 3)
        plt.imshow(maps[i, 150:450, 450:150:-1, 1])
        plt.plot(-robot_poses[:i + 1, 0] / 0.07 + 150, -robot_poses[:i + 1, 1] / 0.07 + 150, color='r')
        plot_img_np = get_img_from_fig(fig)
        frames.append(plot_img_np)
    return frames

In [8]:
runner = HabitatRunner()
for i in runner.eval_episodes[:1]:
    runner.run_episode(i)
    maps = np.array(runner.agent.maps)
    rgbs = np.array(runner.agent.rgbs)
    depths = np.array(runner.agent.depths)
    robot_poses = np.array(runner.agent.robot_pose_track)
    goal_coords = np.array(runner.agent.goal_coords)
    action_track = runner.agent.action_track

2023-04-06 06:34:24,650 Initializing dataset ObjectNav-v1


{'habitat': {'environment': {'max_episode_steps': 500, 'max_episode_seconds': 1500, 'iterator_options': {'shuffle': False}}, 'seed': 123, 'simulator': {'type': 'Sim-v0', 'turn_angle': 30, 'tilt_angle': 30, 'forward_step_size': 0.25, 'action_space_config': 'v1', 'agents': {'main_agent': {'sim_sensors': {'rgb_sensor': {'width': 640, 'height': 480, 'hfov': 79, 'position': [0, 0.88, 0]}, 'depth_sensor': {'width': 640, 'height': 480, 'hfov': 79, 'min_depth': 0.5, 'max_depth': 5.0, 'position': [0, 0.88, 0], 'normalize_depth': False}, 'semantic_sensor': {'width': 640, 'height': 480}}, 'height': 0.88, 'radius': 0.18}}, 'habitat_sim_v0': {'gpu_device_id': 0, 'gpu_gpu': False, 'allow_sliding': False}, 'scene': 'data/scene_datasets/habitat-test-scenes/van-gogh-room.glb', 'scene_dataset': '/data/scene_datasets/hm3d/hm3d_annotated_basis.scene_dataset_config.json', 'default_agent_id': 0, 'additional_object_paths': [], 'agents_order': ['main_agent']}, 'dataset': {'type': 'ObjectNav-v1', 'split': 'val

2023-04-06 06:34:27,065 initializing sim Sim-v0


{'type': 'Sim-v0', 'turn_angle': 30, 'tilt_angle': 30, 'forward_step_size': 0.25, 'action_space_config': 'v1', 'agents': {'main_agent': {'sim_sensors': {'rgb_sensor': {'width': 640, 'height': 480, 'hfov': 79, 'position': [0, 0.88, 0]}, 'depth_sensor': {'width': 640, 'height': 480, 'hfov': 79, 'min_depth': 0.5, 'max_depth': 5.0, 'position': [0, 0.88, 0], 'normalize_depth': False}, 'semantic_sensor': {'width': 640, 'height': 480}}, 'height': 0.88, 'radius': 0.18}}, 'habitat_sim_v0': {'gpu_device_id': 0, 'gpu_gpu': False, 'allow_sliding': False}, 'scene': 'data/scene_datasets/habitat-test-scenes/van-gogh-room.glb', 'scene_dataset': '/data/scene_datasets/hm3d/hm3d_annotated_basis.scene_dataset_config.json', 'default_agent_id': 0, 'additional_object_paths': [], 'agents_order': ['main_agent']}
rgb_sensor {'width': 640, 'height': 480, 'hfov': 79, 'position': [0, 0.88, 0]}
depth_sensor {'width': 640, 'height': 480, 'hfov': 79, 'min_depth': 0.5, 'max_depth': 5.0, 'position': [0, 0.88, 0], 'norm

2023-04-06 06:34:29,836 Initializing task ObjectNav-v1


[06:34:27:079775]:[Metadata] AttributesManagerBase.h(380)::createFromJsonOrDefaultInternal : <Dataset>: Proposing JSON name : default.scene_dataset_config.json from original name : default| This file does not exist.
[06:34:27:079905]:[Metadata] AssetAttributesManager.cpp(123)::createObject : Asset attributes (capsule3DSolid:capsule3DSolid_hemiRings_4_cylRings_1_segments_12_halfLen_0.75_useTexCoords_false_useTangents_false) created and registered.
[06:34:27:079943]:[Metadata] AssetAttributesManager.cpp(123)::createObject : Asset attributes (capsule3DWireframe:capsule3DWireframe_hemiRings_8_cylRings_1_segments_16_halfLen_1) created and registered.
[06:34:27:079960]:[Metadata] AssetAttributesManager.cpp(123)::createObject : Asset attributes (coneSolid:coneSolid_segments_12_halfLen_1.25_rings_1_useTexCoords_false_useTangents_false_capEnd_true) created and registered.
[06:34:27:079975]:[Metadata] AssetAttributesManager.cpp(123)::createObject : Asset attributes (coneWireframe:coneWireframe_s

[06:34:27:161119]:[Assets] ResourceManager.cpp(355)::loadSemanticSceneDescriptor : SSD File Naming Issue! Neither SceneInstanceAttributes-provided name : /data/scene_datasets/hm3d_v0.2/val/00831-yr17PDCnDDW/yr17PDCnDDW.basis.scn  nor constructed filename : /data/scene_datasets/hm3d_v0.2/val/00831-yr17PDCnDDW/info_semantic.json exist on disk.
[06:34:29:630446]:[Sim] Simulator.cpp(478)::instanceStageForSceneAttributes : 
---
The active scene does not contain semantic annotations : activeSemanticSceneID_ = 0  
---


Use load_from_local loader
CLASSES: ('wall', 'building', 'sky', 'floor', 'tree', 'ceiling', 'road', 'bed ', 'windowpane', 'grass', 'cabinet', 'sidewalk', 'person', 'earth', 'door', 'table', 'mountain', 'plant', 'curtain', 'chair', 'car', 'water', 'painting', 'sofa', 'shelf', 'house', 'sea', 'mirror', 'rug', 'field', 'armchair', 'seat', 'fence', 'desk', 'rock', 'wardrobe', 'lamp', 'bathtub', 'railing', 'cushion', 'base', 'box', 'column', 'signboard', 'chest of drawers', 'counter', 'sand', 'sink', 'skyscraper', 'fireplace', 'refrigerator', 'grandstand', 'path', 'stairs', 'runway', 'case', 'pool table', 'pillow', 'screen door', 'stairway', 'river', 'bridge', 'bookcase', 'blind', 'coffee table', 'toilet', 'flower', 'book', 'hill', 'bench', 'countertop', 'stove', 'palm', 'kitchen island', 'computer', 'swivel chair', 'boat', 'bar', 'arcade machine', 'hovel', 'bus', 'towel', 'light', 'truck', 'tower', 'chandelier', 'awning', 'streetlight', 'booth', 'television receiver', 'airplane', 'dirt tra

In [29]:
if not os.path.exists(gif_save_dir):
    os.mkdir(gif_save_dir)
episode_number = 3
objectgoal = 'plant'
frame.write_gif(os.path.join(gif_save_dir, 'episode_{}_{}.gif'.format(episode_number, objectgoal)))

MoviePy - Building file fbe_results/attempt1/episode_3_plant.gif with imageio.


In [9]:
frames = make_gif(maps, rgbs, depths, robot_poses, goal_coords, action_track, i, runner.objectgoal)
frame = ImageSequenceClip(frames, fps=5)
if not os.path.exists(gif_save_dir):
    os.mkdir(gif_save_dir)
frame.write_gif(os.path.join(gif_save_dir, 'episode_{}_{}.gif'.format(episode_number, objectgoal)))
gc.collect()

  0%|          | 0/499 [00:00<?, ?it/s]

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



TypeError: must be real number, not NoneType